# **Semantic AI Übung 9 - Exercise 2**

**Magdalena König 01455794**

**Implementation in Python using `pgmpy`**


In [ ]:
!pip install -q pgmpy networkx matplotlib plotly daft

**Import the required libraries**

In [ ]:
import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
import networkx as nx
import matplotlib.pyplot as plt


**a.) Create the Bayesian network for Johanna and Peter**

In [ ]:
# create the DAG
model = BayesianNetwork([
    ('PetersAlarmFails', 'PeterLate'), 
    ('TrainStrike', 'PeterLate'), 
    ('TrainStrike', 'JohannaLate')])

statenames = {
'TrainStrike': ['True', 'False'],
'PeterLate': ['True', 'False'],
'JohannaLate': ['True', 'False'],
'PetersAlarmFails': ['True', 'False']}

model.to_daft().render()

# for each variable add a tabular conditional probability distribution 
model.add_cpds(
  TabularCPD(
      variable='TrainStrike', 
      variable_card=2,
      state_names=statenames,
      values=[[0.05], [0.95]]),
  TabularCPD(
      variable='PetersAlarmFails', 
      variable_card=2,
      state_names=statenames,
      values=[[0.1], [0.9]]),
  TabularCPD(
      variable='PeterLate', 
      variable_card=2,
      state_names=statenames,
      evidence=['TrainStrike', 'PetersAlarmFails'], 
      evidence_card=[2,2],
      values = [[0.7, 0.3, 0.4, 0.01],
                [0.3, 0.7, 0.6, 0.99]]),
  
  TabularCPD(
      variable='JohannaLate', 
      variable_card=2,
      evidence=['TrainStrike'], 
      evidence_card=[2],
      state_names=statenames,
      values = [[0.5, 0.02],
                [0.5, 0.98]])
)

# print the conditional probability distributions
for cpd in model.get_cpds():
  print(cpd)

**b.) Probability of Johanna/Peter being late**

In [ ]:
from pgmpy.inference import VariableElimination
inference = VariableElimination(model)
result_peter = inference.query(variables=['PeterLate'])
result_johanna = inference.query(variables=['JohannaLate'])
print("Peter late:")
print(result_peter)
print("Johanna late:")
print(result_johanna)

**c.) Johanna is late when Peter's alarm fails**

In [ ]:
evidence = {'PetersAlarmFails':'True'}
result_johanna = inference.query(variables=['JohannaLate'], evidence=evidence)
print("Johanna is late when Peter's alarm fails:",evidence)
print(result_johanna)

**d.) Johanna being late when Peter is** 

In [ ]:
evidence = {'PeterLate':'True'}
result_johanna = inference.query(variables=['JohannaLate'], evidence=evidence)
print("Johanna is late when Peter is:",evidence)
print(result_johanna)

**e.) Johanna being late when Peter is even if his alarm works**

In [ ]:
evidence = {'PeterLate':'True', 'PetersAlarmFails':'False'}
result_johanna = inference.query(variables=['JohannaLate'], evidence=evidence)
print("Johanna is late when Peter is despite his alarm working:",evidence)
print(result_johanna)